# TSA_ch8_acf_comparison

## ACF Comparison: Short Memory vs Long Memory

**Data**: EUR/RON Exchange Rate (BNR)

**Author**: Daniel Traian PELE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Load Real Data: EUR/RON Exchange Rate

In [ ]:
# Download EUR/RON data from Yahoo Finance
ticker = 'EURRON=X'
df = yf.download(ticker, start='2010-01-01', end='2024-12-31', progress=False)

# Use closing price
eurron = df['Close'].dropna()
print(f"Data period: {eurron.index[0].date()} to {eurron.index[-1].date()}")
print(f"Observations: {len(eurron)}")

In [ ]:
# Calculate returns and squared returns
returns = np.log(eurron / eurron.shift(1)).dropna() * 100  # Log returns in %
squared_returns = returns ** 2

print(f"Returns - Mean: {returns.mean():.4f}%, Std: {returns.std():.4f}%")
print(f"Squared returns - Mean: {squared_returns.mean():.4f}")

## 2. ACF Comparison: Returns vs Squared Returns

In [ ]:
n_lags = 100

# Compute ACF
acf_returns = acf(returns, nlags=n_lags, fft=True)
acf_squared = acf(squared_returns, nlags=n_lags, fft=True)

# Confidence bands (95%)
conf_band = 1.96 / np.sqrt(len(returns))

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Returns series
axes[0, 0].plot(returns.index, returns.values, color='steelblue', linewidth=0.5)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=0.5)
axes[0, 0].set_title('EUR/RON Log Returns (%)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Return (%)')

# Plot 2: Squared returns series
axes[0, 1].plot(squared_returns.index, squared_returns.values, color='darkred', linewidth=0.5)
axes[0, 1].set_title('EUR/RON Squared Returns (Volatility Proxy)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Squared Return')

# Plot 3: ACF of returns (SHORT MEMORY)
axes[1, 0].bar(range(n_lags + 1), acf_returns, color='steelblue', width=0.8)
axes[1, 0].axhline(y=conf_band, color='red', linestyle='--', linewidth=1)
axes[1, 0].axhline(y=-conf_band, color='red', linestyle='--', linewidth=1)
axes[1, 0].axhline(y=0, color='black', linewidth=0.5)
axes[1, 0].set_title('ACF of Returns - SHORT MEMORY', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('Autocorrelation')
axes[1, 0].set_ylim(-0.15, 0.15)

# Plot 4: ACF of squared returns (LONG MEMORY)
axes[1, 1].bar(range(n_lags + 1), acf_squared, color='darkred', width=0.8)
axes[1, 1].axhline(y=conf_band, color='blue', linestyle='--', linewidth=1)
axes[1, 1].axhline(y=-conf_band, color='blue', linestyle='--', linewidth=1)
axes[1, 1].axhline(y=0, color='black', linewidth=0.5)
axes[1, 1].set_title('ACF of Squared Returns - LONG MEMORY', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('Autocorrelation')

plt.tight_layout()
plt.savefig('ch8_acf_comparison.pdf', bbox_inches='tight', dpi=150)
plt.show()

## 3. Key Insight: Memory in Financial Data

**Stylized Fact**: 
- **Returns** have SHORT memory (ACF decays quickly to zero)
- **Squared returns (volatility)** have LONG memory (ACF decays slowly)

In [ ]:
# Quantify the difference
print("="*60)
print("Memory Comparison: Returns vs Squared Returns")
print("="*60)
print(f"\nACF at lag 10:  Returns = {acf_returns[10]:.4f}, Squared = {acf_squared[10]:.4f}")
print(f"ACF at lag 50:  Returns = {acf_returns[50]:.4f}, Squared = {acf_squared[50]:.4f}")
print(f"ACF at lag 100: Returns = {acf_returns[100]:.4f}, Squared = {acf_squared[100]:.4f}")
print(f"\nSum of |ACF| (lags 1-100):")
print(f"  Returns:         {np.sum(np.abs(acf_returns[1:])):.4f}")
print(f"  Squared Returns: {np.sum(np.abs(acf_squared[1:])):.4f}")
print("\n=> Squared returns show much stronger persistence (long memory)!")